# Manipulação de Dados

Considere o seguinte conjunto de dados textuais:
- O gato está no telhado.
- A chuva cai sem parar.
- Gosto de assistir filmes nos finais de semana.
- Ele está estudando para as provas finais.

> Realize a limpeza dos textos, incluindo a remoção de stopwords, normalização (caixa baixa), e tokenização.

> Aplique lematização aos textos.

> Crie vetores de característica utilizando TF-IDF e Bag of Words.


In [ ]:
!pip install -r requirements.txt

### Início: Limpeza dos textos

Inicialmente, vou instanciar a lista com os textos. Cada um é como um documento (futuramente será assim feito o TF-IDF).

In [1]:
batches = [
    "O gato está no telhado.",
    "A chuva cai sem parar.",
    "Gosto de assistir filmes nos finais de semana.",
    "Ele está estudando para as provas finais."
]

batches

['O gato está no telhado.',
 'A chuva cai sem parar.',
 'Gosto de assistir filmes nos finais de semana.',
 'Ele está estudando para as provas finais.']

#### Adaptação do lote (batches)

Para a formatação do texto e remoção das pontuações, utilizarei métodos padrões do python visto que esses facilmente dão conta do serviço. Irei utilizar o Spacy para remover as stopwords, realizar a tokenização e a lemetização visto que ele já possui métodos para fazer isso com português brasileiro de maneira eficiente.

A ordem das minhas ações será:
- Normalização
- Remoção de pontuações
- Remoção de stopwords, tokenização e lemetização

In [2]:
batches = [phrase.lower() for phrase in batches]

batches

['o gato está no telhado.',
 'a chuva cai sem parar.',
 'gosto de assistir filmes nos finais de semana.',
 'ele está estudando para as provas finais.']

In [3]:
# importando para ter acesso rápido às pontuações como string
import string

In [4]:
# Remoção de pontuações

punctuations = string.punctuation
print(f"{punctuations =}")

cleared_texts = []
for phrase in batches:
    words = list(phrase)
    print(f"{words =}")
    cleared_texts.append("".join([word for word in words if word not in punctuations]))

cleared_texts

punctuations ='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
words =['o', ' ', 'g', 'a', 't', 'o', ' ', 'e', 's', 't', 'á', ' ', 'n', 'o', ' ', 't', 'e', 'l', 'h', 'a', 'd', 'o', '.']
words =['a', ' ', 'c', 'h', 'u', 'v', 'a', ' ', 'c', 'a', 'i', ' ', 's', 'e', 'm', ' ', 'p', 'a', 'r', 'a', 'r', '.']
words =['g', 'o', 's', 't', 'o', ' ', 'd', 'e', ' ', 'a', 's', 's', 'i', 's', 't', 'i', 'r', ' ', 'f', 'i', 'l', 'm', 'e', 's', ' ', 'n', 'o', 's', ' ', 'f', 'i', 'n', 'a', 'i', 's', ' ', 'd', 'e', ' ', 's', 'e', 'm', 'a', 'n', 'a', '.']
words =['e', 'l', 'e', ' ', 'e', 's', 't', 'á', ' ', 'e', 's', 't', 'u', 'd', 'a', 'n', 'd', 'o', ' ', 'p', 'a', 'r', 'a', ' ', 'a', 's', ' ', 'p', 'r', 'o', 'v', 'a', 's', ' ', 'f', 'i', 'n', 'a', 'i', 's', '.']


['o gato está no telhado',
 'a chuva cai sem parar',
 'gosto de assistir filmes nos finais de semana',
 'ele está estudando para as provas finais']

In [5]:
import spacy

> Nessa etapa, utilizarei o spacy e suas funções para efetuar simultaneamente tanto a lemetização quanto a remoção de stopwords e tokenização. Isso não é difícil de fazer e poder ser otimizado aplicando-as em uma compreensão de lista (list comprehension).

In [6]:
# Carregando um modelo em português para efetuar o processamento
nlp = spacy.load("pt_core_news_sm")

In [7]:
# Iterando sobre cada frase do lote para executar a sequência de processos
tokens_list = []
for phrase in cleared_texts:
    loaded = nlp(phrase)
    tokens = [token.lemma_ for token in loaded if all([token.pos_ == "VERB", not token.is_stop])]
    tokens_list.append(tokens)

tokens_list

[[], ['cair', 'parar'], ['gostar', 'assistir'], ['estudar']]

#### Explicação

A parte que pode parecer mais complexa dessa etapa é a compreensão de lista, portanto, eis aqui a explicação:
- São feitas 3 validações para cada token ser incluído na lista e lematizado;
- A primeira validação é se o token é um verbo. Isso é feito pois, para a lematização, apenas funcionam verbos, portanto são excluídas palavras não detectadas nessa categoria;
- A segunda validação é a se o token é uma stopword. Caso seja, não é incluído na lista.
- Ao fim, em `token.lemma_`, é inclusa na lista a versão lematizada dos tokens que tenham passado nas validações.

##### Observação

Pode ser observado no exemplo que, mesmo filtrando por verbos, ainda assim alguns dos verbos não apareceram na lista final. Isso pode se dar por dois fatores:
- O MLN não é perfeito e comete erros;
- A palavra é uma stopword.

Em cenários de treinamento ou preparação de documentos para treinamento com lotes bem maiores, estatisticamente os erros tendem a se balancear e não atrapalham o resultado final. 

### Última etapa

Por fim, utilizarei o sklearn para realizar o processo de TF-IDF pois ele já possui funções bem otimizadas e eficientes para executar isso.

Para título de curiosidade, anexo aqui também o cálculo manual do TF-IDF:
> TF = frequencia_do_token / len(tokens)

> IDF = log (len(batches) / len([batch for batch in batches if token in batch]))

> TF-IDF = TF * IDF

Depois utilizarei o Bag of Words para construir e exibir a matriz usando essa técnica. Para isso o Sklearn também possui funções eficientes e otimizadas que fazem os cálculos.


In [46]:
# TfidfVectorizer para cálculo do TF-IDF, CountVectorizer para Bag of Words
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [47]:
batches_in_text = [" ".join(tokens) for tokens in tokens_list]
batches_in_text

['', 'cair parar', 'gostar assistir', 'estudar']

### TF-IDF:

In [52]:
vectorizer = TfidfVectorizer()

# Efetuar a transformação de documentos/strings
vectors = vectorizer.fit_transform(batches_in_text)

# Transformar em lista os TF-IDF e exibir
tfidf = vectors.todense()
tfidf_list = tfidf.tolist()
print(f"{tfidf_list =}")

tfidf_list =[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.7071067811865476, 0.0, 0.0, 0.7071067811865476], [0.7071067811865476, 0.0, 0.0, 0.7071067811865476, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0]]


In [53]:
# Para melhor visualização, transformarei os dados em uma planilha do pandas
import pandas as pd

In [54]:
# Separar os nomes dos tokens
feature_names = list(vectorizer.get_feature_names_out())
print(f"{feature_names =}")

feature_names =['assistir', 'cair', 'estudar', 'gostar', 'parar']


In [55]:
# Transformar em DataFrame
df = pd.DataFrame(tfidf_list, columns=feature_names)

df

,assistir,cair,estudar,gostar,parar
0,0.000000,0.000000,0.0,0.000000,0.000000
1,0.000000,0.707107,0.0,0.000000,0.707107
2,0.707107,0.000000,0.0,0.707107,0.000000
3,0.000000,0.000000,1.0,0.000000,0.000000


### Bag of Words:

In [65]:
bag_of_words = CountVectorizer()

In [73]:
matrix = bag_of_words.fit_transform(batches_in_text)
matrix.toarray()

array([[0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0]])

In [78]:
# Transformando em DataFrame do pandas
feature_names = list(bag_of_words.get_feature_names_out())

df = pd.DataFrame(matrix.toarray(), columns=feature_names)

df

,assistir,cair,estudar,gostar,parar
0,0,0,0,0,0
1,0,1,0,0,1
2,1,0,0,1,0
3,0,0,1,0,0
